# **How well do jobs and skills pay for Data Analysts?**

### **Methology**
1. Evaluate median salary for top 6 data jobs
2. Find median salary per skill for Data Analysts
3. Visualize for highest paying skills and most demanded skills

### **Original Exploration**
[18_Matplotlib_Box_Plots.ipynb](https://github.com/adinramaadin/Python_Project-2_Data-Job-Market/blob/main/Exploring_the_Data/17_Matplotlib_Box_Plots.ipynb)

[19_Exercise_Skill_Pay_Analysis.ipynb](https://github.com/adinramaadin/Python_Project-2_Data-Job-Market/blob/main/Exploring_the_Data/19_Seaborn.ipynb)
